# Load Library

In [8]:
import os
import json
import csv
import pandas as pd
import datasets

from glob import glob
from tqdm import tqdm
from huggingface_hub import notebook_login, create_tag
from datasets import Dataset, DatasetDict, load_dataset

In [2]:
os.getcwd()

'/data/heerak/workspace/translation_koenja'

In [3]:
data_path = os.path.join(os.getcwd(), 'data')

In [4]:
data_path

'/data/heerak/workspace/translation_koenja/data'

In [5]:
glob(data_path + '/*')

['/data/heerak/workspace/translation_koenja/data/jp_ko',
 '/data/heerak/workspace/translation_koenja/data/ko_jp',
 '/data/heerak/workspace/translation_koenja/data/train',
 '/data/heerak/workspace/translation_koenja/data/ko_en',
 '/data/heerak/workspace/translation_koenja/data/validation',
 '/data/heerak/workspace/translation_koenja/data/en_ko']

# Make parallel corpus folder

In [71]:
if not os.path.exists(os.path.join(data_path, 'en_ko')) or not os.path.exists(os.path.join(data_path, 'ko_en')) or \
    not os.path.exists(os.path.join(data_path, 'ko_jp')) or not os.path.exists(os.path.join(data_path, 'jp_ko')):
    os.makedirs(os.path.join(data_path, 'en_ko'), exist_ok=True)
    os.makedirs(os.path.join(data_path, 'ko_en'), exist_ok=True)
    os.makedirs(os.path.join(data_path, 'jp_ko'), exist_ok=True)
    os.makedirs(os.path.join(data_path, 'ko_jp'), exist_ok=True)

# Preprocessing

## 1. 025.일상생활 및 구어체 한-영 번역 병렬 말뭉치 데이터

### 1.1 en-ko train data

In [72]:
file_list = sorted(glob(os.path.join(data_path, 'train', '025.일상생활 및 구어체 한-영 번역 병렬 말뭉치 데이터/*영한*')))
file_list

['/data/heerak/workspace/translation_koenja/data/train/025.일상생활 및 구어체 한-영 번역 병렬 말뭉치 데이터/일상생활및구어체_영한_train_set.json']

In [73]:
for fname in file_list:
    print(fname)
    source = 'en'
    target = 'ko'
    source_target = f'{source}_{target}'
    mode = 'train'
    with open(fname, 'r') as json_file:
        json_data = json.load(json_file)
        
    if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')):
        with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 'r') as f:
            cnt = max(len(f.readlines()) - 1, 0)
    else:
        cnt = 0
    
    with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 
              'a' if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')) else 'w') as f:
        if cnt == 0:
            f.write(f'{source}\t{target}')
        for data in tqdm(json_data['data'], desc=f"{fname.split('/')[-1]}"):
            en = data['en'].replace('>', '').replace('\n', '').strip()
            ko = data['ko'].replace('>', '').replace('\n', '').strip()
            f.write(f'\n{en}\t{ko}')
            cnt += 1
    
    print(f'총 {cnt} 문장')

/data/heerak/workspace/translation_koenja/data/train/025.일상생활 및 구어체 한-영 번역 병렬 말뭉치 데이터/일상생활및구어체_영한_train_set.json


일상생활및구어체_영한_train_set.json: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1200307/1200307 [00:01<00:00, 901127.66it/s]

총 1200307 문장


### 1.2 ko-en train data

In [74]:
file_list = sorted(glob(os.path.join(data_path, 'train', '025.일상생활 및 구어체 한-영 번역 병렬 말뭉치 데이터/*한영*')))
file_list

['/data/heerak/workspace/translation_koenja/data/train/025.일상생활 및 구어체 한-영 번역 병렬 말뭉치 데이터/일상생활및구어체_한영_train_set.json']

In [75]:
for fname in file_list:
    print(fname)
    source = 'ko'
    target = 'en'
    source_target = f'{source}_{target}'
    mode = 'train'
    with open(fname, 'r') as json_file:
        json_data = json.load(json_file)
        
    if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')):
        with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 'r') as f:
            cnt = max(len(f.readlines()) - 1, 0)
    else:
        cnt = 0
    
    with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 
              'a' if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')) else 'w') as f:
        if cnt == 0:
            f.write(f'{source}\t{target}')
        for data in tqdm(json_data['data'], desc=f"{fname.split('/')[-1]}"):
            en = data['en'].replace('>', '').replace('\n', '').strip()
            ko = data['ko'].replace('>', '').replace('\n', '').strip()
            f.write(f'\n{ko}\t{en}')
            cnt += 1
    
    print(f'총 {cnt} 문장')

/data/heerak/workspace/translation_koenja/data/train/025.일상생활 및 구어체 한-영 번역 병렬 말뭉치 데이터/일상생활및구어체_한영_train_set.json


일상생활및구어체_한영_train_set.json: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1200000/1200000 [00:01<00:00, 870869.46it/s]

총 1200000 문장


### 1.3 en-ko validation data

In [76]:
file_list = sorted(glob(os.path.join(data_path, 'validation', '025.일상생활 및 구어체 한-영 번역 병렬 말뭉치 데이터/*영한*')))
file_list

['/data/heerak/workspace/translation_koenja/data/validation/025.일상생활 및 구어체 한-영 번역 병렬 말뭉치 데이터/일상생활및구어체_영한_valid_set.json']

In [77]:
for fname in file_list:
    print(fname)
    source = 'en'
    target = 'ko'
    source_target = f'{source}_{target}'
    mode = 'valid'
    with open(fname, 'r') as json_file:
        json_data = json.load(json_file)
        
    if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')):
        with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 'r') as f:
            cnt = max(len(f.readlines()) - 1, 0)
    else:
        cnt = 0
    
    with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 
              'a' if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')) else 'w') as f:
        if cnt == 0:
            f.write(f'{source}\t{target}')
        for data in tqdm(json_data['data'], desc=f"{fname.split('/')[-1]}"):
            en = data['en'].replace('>', '').replace('\n', '').strip()
            ko = data['ko'].replace('>', '').replace('\n', '').strip()
            f.write(f'\n{en}\t{ko}')
            cnt += 1
    
    print(f'총 {cnt} 문장')

/data/heerak/workspace/translation_koenja/data/validation/025.일상생활 및 구어체 한-영 번역 병렬 말뭉치 데이터/일상생활및구어체_영한_valid_set.json


일상생활및구어체_영한_valid_set.json: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150038/150038 [00:00<00:00, 834758.17it/s]

총 150038 문장


### 1.4 ko-en validation data

In [78]:
file_list = sorted(glob(os.path.join(data_path, 'validation', '025.일상생활 및 구어체 한-영 번역 병렬 말뭉치 데이터/*한영*')))
file_list

['/data/heerak/workspace/translation_koenja/data/validation/025.일상생활 및 구어체 한-영 번역 병렬 말뭉치 데이터/일상생활및구어체_한영_valid_set.json']

In [79]:
for fname in file_list:
    print(fname)
    source = 'ko'
    target = 'en'
    source_target = f'{source}_{target}'
    mode = 'valid'
    with open(fname, 'r') as json_file:
        json_data = json.load(json_file)
        
    if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')):
        with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 'r') as f:
            cnt = max(len(f.readlines()) - 1, 0)
    else:
        cnt = 0
    
    with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 
              'a' if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')) else 'w') as f:
        if cnt == 0:
            f.write(f'{source}\t{target}')
        for data in tqdm(json_data['data'], desc=f"{fname.split('/')[-1]}"):
            en = data['en'].replace('>', '').replace('\n', '').strip()
            ko = data['ko'].replace('>', '').replace('\n', '').strip()
            f.write(f'\n{ko}\t{en}')
            cnt += 1
    
    print(f'총 {cnt} 문장')

/data/heerak/workspace/translation_koenja/data/validation/025.일상생활 및 구어체 한-영 번역 병렬 말뭉치 데이터/일상생활및구어체_한영_valid_set.json


일상생활및구어체_한영_valid_set.json: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150000/150000 [00:00<00:00, 859726.95it/s]

총 150000 문장


## 2. 026.기술과학 분야 한-영 번역 병렬 말뭉치 데이터

### 2.1 ko-en train data

In [80]:
file_list = sorted(glob(os.path.join(data_path, 'train', '026.기술과학 분야 한-영 번역 병렬 말뭉치 데이터/*')))
file_list

['/data/heerak/workspace/translation_koenja/data/train/026.기술과학 분야 한-영 번역 병렬 말뭉치 데이터/기술과학분야_한영_train_set.json']

In [81]:
for fname in file_list:
    print(fname)
    source = 'ko'
    target = 'en'
    source_target = f'{source}_{target}'
    mode = 'train'
    with open(fname, 'r') as json_file:
        json_data = json.load(json_file)
        
    if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')):
        with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 'r') as f:
            cnt = max(len(f.readlines()) - 1, 0)
    else:
        cnt = 0
    
    with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 
              'a' if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')) else 'w') as f:
        if cnt == 0:
            f.write(f'{source}\t{target}')
        for data in tqdm(json_data['data'], desc=f"{fname.split('/')[-1]}"):
            en = data['en'].replace('>', '').replace('\n', '').strip()
            ko = data['ko'].replace('>', '').replace('\n', '').strip()
            f.write(f'\n{ko}\t{en}')
            cnt += 1
    
    print(f'총 {cnt} 문장')

/data/heerak/workspace/translation_koenja/data/train/026.기술과학 분야 한-영 번역 병렬 말뭉치 데이터/기술과학분야_한영_train_set.json


기술과학분야_한영_train_set.json: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1200144/1200144 [00:02<00:00, 527201.32it/s]

총 2400144 문장


### 2.2 ko-en validation data

In [82]:
file_list = sorted(glob(os.path.join(data_path, 'validation', '026.기술과학 분야 한-영 번역 병렬 말뭉치 데이터/*')))
file_list

['/data/heerak/workspace/translation_koenja/data/validation/026.기술과학 분야 한-영 번역 병렬 말뭉치 데이터/기술과학분야_한영_valid_set.json']

In [83]:
for fname in file_list:
    print(fname)
    source = 'ko'
    target = 'en'
    source_target = f'{source}_{target}'
    mode = 'valid'
    with open(fname, 'r') as json_file:
        json_data = json.load(json_file)
        
    if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')):
        with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 'r') as f:
            cnt = max(len(f.readlines()) - 1, 0)
    else:
        cnt = 0
    
    with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 
              'a' if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')) else 'w') as f:
        if cnt == 0:
            f.write(f'{source}\t{target}')
        for data in tqdm(json_data['data'], desc=f"{fname.split('/')[-1]}"):
            en = data['en'].replace('>', '').replace('\n', '').strip()
            ko = data['ko'].replace('>', '').replace('\n', '').strip()
            f.write(f'\n{ko}\t{en}')
            cnt += 1
    
    print(f'총 {cnt} 문장')

/data/heerak/workspace/translation_koenja/data/validation/026.기술과학 분야 한-영 번역 병렬 말뭉치 데이터/기술과학분야_한영_valid_set.json


기술과학분야_한영_valid_set.json: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150018/150018 [00:00<00:00, 529851.21it/s]

총 300018 문장


## 3. 027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터

### 3.1 ko-jp train data

In [84]:
file_list = sorted(glob(os.path.join(data_path, 'train', '027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터/*한일*')))
file_list

['/data/heerak/workspace/translation_koenja/data/train/027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터/라벨링데이터_한일_일상생활_TTA품질검증_250000_training.json',
 '/data/heerak/workspace/translation_koenja/data/train/027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터/라벨링데이터_한일_채팅_TTA품질검증_150000_training.json',
 '/data/heerak/workspace/translation_koenja/data/train/027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터/라벨링데이터_한일_해외영업_TTA품질검증_350000_training.json']

In [85]:
for fname in file_list:
    print(fname)
    source = 'ko'
    target = 'jp'
    source_target = f'{source}_{target}'
    mode = 'train'
    with open(fname, 'r') as json_file:
        json_data = json.load(json_file)
        
    if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')):
        with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 'r') as f:
            cnt = max(len(f.readlines()) - 1, 0)
    else:
        cnt = 0
    
    with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 
              'a' if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')) else 'w') as f:
        if cnt == 0:
            f.write(f'{source}\t{target}')
        for data in tqdm(json_data, desc=f"{fname.split('/')[-1]}"):
            ko = data['원문'].replace('>', '').replace('\n', '').strip()
            jp = data['최종번역문'].replace('>', '').replace('\n', '').strip()
            f.write(f'\n{ko}\t{jp}')
            cnt += 1
    
    print(f'총 {cnt} 문장')

/data/heerak/workspace/translation_koenja/data/train/027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터/라벨링데이터_한일_일상생활_TTA품질검증_250000_training.json


라벨링데이터_한일_일상생활_TTA품질검증_250000_training.json: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200000/200000 [00:00<00:00, 1034325.58it/s]


총 200000 문장
/data/heerak/workspace/translation_koenja/data/train/027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터/라벨링데이터_한일_채팅_TTA품질검증_150000_training.json


라벨링데이터_한일_채팅_TTA품질검증_150000_training.json: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 120000/120000 [00:00<00:00, 996100.20it/s]


총 320000 문장
/data/heerak/workspace/translation_koenja/data/train/027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터/라벨링데이터_한일_해외영업_TTA품질검증_350000_training.json


라벨링데이터_한일_해외영업_TTA품질검증_350000_training.json: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 280000/280000 [00:00<00:00, 991685.99it/s]

총 600000 문장


### 3.2 ko-jp validation data

In [86]:
file_list = sorted(glob(os.path.join(data_path, 'validation', '027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터/*한일*')))
file_list

['/data/heerak/workspace/translation_koenja/data/validation/027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터/라벨링데이터_한일_일상생활_TTA품질검증_250000_validation.json',
 '/data/heerak/workspace/translation_koenja/data/validation/027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터/라벨링데이터_한일_채팅_TTA품질검증_150000_validation.json',
 '/data/heerak/workspace/translation_koenja/data/validation/027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터/라벨링데이터_한일_해외영업_TTA품질검증_350000_validation.json']

In [87]:
for fname in file_list:
    print(fname)
    source = 'ko'
    target = 'jp'
    source_target = f'{source}_{target}'
    mode = 'valid'
    with open(fname, 'r') as json_file:
        json_data = json.load(json_file)
        
    if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')):
        with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 'r') as f:
            cnt = max(len(f.readlines()) - 1, 0)
    else:
        cnt = 0
    
    with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 
              'a' if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')) else 'w') as f:
        if cnt == 0:
            f.write(f'{source}\t{target}')
        for data in tqdm(json_data, desc=f"{fname.split('/')[-1]}"):
            ko = data['원문'].replace('>', '').replace('\n', '').strip()
            jp = data['최종번역문'].replace('>', '').replace('\n', '').strip()
            f.write(f'\n{ko}\t{jp}')
            cnt += 1
    
    print(f'총 {cnt} 문장')

/data/heerak/workspace/translation_koenja/data/validation/027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터/라벨링데이터_한일_일상생활_TTA품질검증_250000_validation.json


라벨링데이터_한일_일상생활_TTA품질검증_250000_validation.json: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25000/25000 [00:00<00:00, 1006919.73it/s]


총 25000 문장
/data/heerak/workspace/translation_koenja/data/validation/027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터/라벨링데이터_한일_채팅_TTA품질검증_150000_validation.json


라벨링데이터_한일_채팅_TTA품질검증_150000_validation.json: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15000/15000 [00:00<00:00, 963130.29it/s]


총 40000 문장
/data/heerak/workspace/translation_koenja/data/validation/027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터/라벨링데이터_한일_해외영업_TTA품질검증_350000_validation.json


라벨링데이터_한일_해외영업_TTA품질검증_350000_validation.json: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35000/35000 [00:00<00:00, 961706.43it/s]

총 75000 문장


### 3.3 jp-ko train data

In [88]:
file_list = sorted(glob(os.path.join(data_path, 'train', '027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터/*일한*')))
file_list

['/data/heerak/workspace/translation_koenja/data/train/027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터/라벨링데이터_일한_일상생활_TTA품질검증_250000_training.json',
 '/data/heerak/workspace/translation_koenja/data/train/027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터/라벨링데이터_일한_채팅_TTA품질검증_150000_training.json',
 '/data/heerak/workspace/translation_koenja/data/train/027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터/라벨링데이터_일한_해외영업_TTA품질검증_350000_training.json']

In [89]:
for fname in file_list:
    print(fname)
    source = 'jp'
    target = 'ko'
    source_target = f'{source}_{target}'
    mode = 'train'
    with open(fname, 'r') as json_file:
        json_data = json.load(json_file)
        
    if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')):
        with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 'r') as f:
            cnt = max(len(f.readlines()) - 1, 0)
    else:
        cnt = 0
    
    with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 
              'a' if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')) else 'w') as f:
        if cnt == 0:
            f.write(f'{source}\t{target}')
        for data in tqdm(json_data, desc=f"{fname.split('/')[-1]}"):
            jp = data['원문'].replace('>', '').replace('\n', '').strip()
            ko = data['최종번역문'].replace('>', '').replace('\n', '').strip()
            f.write(f'\n{jp}\t{ko}')
            cnt += 1
    
    print(f'총 {cnt} 문장')

/data/heerak/workspace/translation_koenja/data/train/027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터/라벨링데이터_일한_일상생활_TTA품질검증_250000_training.json


라벨링데이터_일한_일상생활_TTA품질검증_250000_training.json: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200000/200000 [00:00<00:00, 993002.61it/s]


총 200000 문장
/data/heerak/workspace/translation_koenja/data/train/027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터/라벨링데이터_일한_채팅_TTA품질검증_150000_training.json


라벨링데이터_일한_채팅_TTA품질검증_150000_training.json: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 120000/120000 [00:00<00:00, 1002111.43it/s]


총 320000 문장
/data/heerak/workspace/translation_koenja/data/train/027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터/라벨링데이터_일한_해외영업_TTA품질검증_350000_training.json


라벨링데이터_일한_해외영업_TTA품질검증_350000_training.json: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 280000/280000 [00:00<00:00, 966931.58it/s]

총 600000 문장


### 3.3 jp-ko validation data

In [90]:
file_list = sorted(glob(os.path.join(data_path, 'validation', '027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터/*일한*')))
file_list

['/data/heerak/workspace/translation_koenja/data/validation/027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터/라벨링데이터_일한_일상생활_TTA품질검증_250000_validation.json',
 '/data/heerak/workspace/translation_koenja/data/validation/027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터/라벨링데이터_일한_채팅_TTA품질검증_150000_validation.json',
 '/data/heerak/workspace/translation_koenja/data/validation/027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터/라벨링데이터_일한_해외영업_TTA품질검증_350000_validation.json']

In [91]:
for fname in file_list:
    print(fname)
    source = 'jp'
    target = 'ko'
    source_target = f'{source}_{target}'
    mode = 'valid'
    with open(fname, 'r') as json_file:
        json_data = json.load(json_file)
        
    if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')):
        with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 'r') as f:
            cnt = max(len(f.readlines()) - 1, 0)
    else:
        cnt = 0
    
    with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 
              'a' if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')) else 'w') as f:
        if cnt == 0:
            f.write(f'{source}\t{target}')
        for data in tqdm(json_data, desc=f"{fname.split('/')[-1]}"):
            jp = data['원문'].replace('>', '').replace('\n', '').strip()
            ko = data['최종번역문'].replace('>', '').replace('\n', '').strip()
            f.write(f'\n{jp}\t{ko}')
            cnt += 1
    
    print(f'총 {cnt} 문장')

/data/heerak/workspace/translation_koenja/data/validation/027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터/라벨링데이터_일한_일상생활_TTA품질검증_250000_validation.json


라벨링데이터_일한_일상생활_TTA품질검증_250000_validation.json: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 25000/25000 [00:00<00:00, 935944.45it/s]


총 25000 문장
/data/heerak/workspace/translation_koenja/data/validation/027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터/라벨링데이터_일한_채팅_TTA품질검증_150000_validation.json


라벨링데이터_일한_채팅_TTA품질검증_150000_validation.json: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15000/15000 [00:00<00:00, 951103.72it/s]


총 40000 문장
/data/heerak/workspace/translation_koenja/data/validation/027.일상생활 및 구어체 한-중, 한-일 번역 병렬 말뭉치 데이터/라벨링데이터_일한_해외영업_TTA품질검증_350000_validation.json


라벨링데이터_일한_해외영업_TTA품질검증_350000_validation.json: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 35000/35000 [00:00<00:00, 919319.66it/s]

총 75000 문장


## 4. 155.산업정보 연계 주요국 특허 영-한 데이터

### 4.1 en-ko train data

In [92]:
file_list = sorted(glob(os.path.join(data_path, 'train', '155.산업정보 연계 주요국 특허 영-한 데이터/*')))
file_list, len(file_list)

(['/data/heerak/workspace/translation_koenja/data/train/155.산업정보 연계 주요국 특허 영-한 데이터/1ь░и ъ╕ИьЖНьаЬэТИ ы░П ъ╕ИьЖНъ┤Сым╝ ыПДыздьЧЕ.json',
  '/data/heerak/workspace/translation_koenja/data/train/155.산업정보 연계 주요국 특허 영-한 데이터/ъ│Дый┤эЩЬьД▒ьаЬ ьаЬьб░ьЧЕ.json',
  '/data/heerak/workspace/translation_koenja/data/train/155.산업정보 연계 주요국 특허 영-한 데이터/ъ│аым┤ ьЭШыеШ ы░П ъ╕░эГА ьЬДьГЭьЪй ы╣Дъ▓╜эЩФ ъ│аым┤ьаЬэТИ ьаЬьб░ьЧЕ.json',
  '/data/heerak/workspace/translation_koenja/data/train/155.산업정보 연계 주요국 특허 영-한 데이터/ъ│аым┤ эГАьЭ┤ьЦ┤ ы░П эКЬы╕М ьГЭьВ░ьЧЕ.json',
  '/data/heerak/workspace/translation_koenja/data/train/155.산업정보 연계 주요국 특허 영-한 데이터/ъ│аым┤ эМиэВ╣ыеШ ьаЬьб░ьЧЕ.json',
  '/data/heerak/workspace/translation_koenja/data/train/155.산업정보 연계 주요국 특허 영-한 데이터/ъ│аым┤, эЩФэХЩьДмьЬа ы░П эФМыЭ╝ьКдэЛ▒ ьД▒эШХъ╕░ ьаЬьб░ьЧЕ.json',
  '/data/heerak/workspace/translation_koenja/data/train/155.산업정보 연계 주요국 특허 영-한 데이터/ъ│бым╝ ъ░Аъ│╡эТИ ьаЬьб░ьЧЕ.json',
  '/data/heerak/workspace/translation_koenja/data/train/155.산업정보 연계 주요국 특허 영-한 데이

In [93]:
for fname in file_list:
    print(fname)
    source = 'en'
    target = 'ko'
    source_target = f'{source}_{target}'
    mode = 'train'
    with open(fname, 'r') as json_file:
        json_data = json.load(json_file)
        
    if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')):
        with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 'r') as f:
            cnt = max(len(f.readlines()) - 1, 0)
    else:
        cnt = 0
    
    with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 
              'a' if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')) else 'w') as f:
        if cnt == 0:
            f.write(f'{source}\t{target}')
        for data in tqdm(json_data['labeled_data'], desc=f"{fname.split('/')[-1]}"):
            for key in ['invention_title', 'astrt_cont', 'claim']:
                en = data[f'{key}_eng'].replace('>', '').replace('\n', '').strip()
                ko = data[f'{key}_kor'].replace('>', '').replace('\n', '').strip()
                f.write(f'\n{en}\t{ko}')
                cnt += 1
    
    print(f'총 {cnt} 문장')

/data/heerak/workspace/translation_koenja/data/train/155.산업정보 연계 주요국 특허 영-한 데이터/1ь░и ъ╕ИьЖНьаЬэТИ ы░П ъ╕ИьЖНъ┤Сым╝ ыПДыздьЧЕ.json


1ь░и ъ╕ИьЖНьаЬэТИ ы░П ъ╕ИьЖНъ┤Сым╝ ыПДыздьЧЕ.json: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 23/23 [00:00<00:00, 33991.89it/s]


총 1200376 문장
/data/heerak/workspace/translation_koenja/data/train/155.산업정보 연계 주요국 특허 영-한 데이터/ъ│Дый┤эЩЬьД▒ьаЬ ьаЬьб░ьЧЕ.json


ъ│Дый┤эЩЬьД▒ьаЬ ьаЬьб░ьЧЕ.json: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 618/618 [00:00<00:00, 97150.78it/s]


총 1202230 문장
/data/heerak/workspace/translation_koenja/data/train/155.산업정보 연계 주요국 특허 영-한 데이터/ъ│аым┤ ьЭШыеШ ы░П ъ╕░эГА ьЬДьГЭьЪй ы╣Дъ▓╜эЩФ ъ│аым┤ьаЬэТИ ьаЬьб░ьЧЕ.json


...

총 2159824 문장
/data/heerak/workspace/translation_koenja/data/train/155.산업정보 연계 주요국 특허 영-한 데이터/эЩШъ▓╜ьДды╣Д ъ▒┤ьДдьЧЕ.json


эЩШъ▓╜ьДды╣Д ъ▒┤ьДдьЧЕ.json: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 138/138 [00:00<00:00, 79968.77it/s]


총 2160238 문장
/data/heerak/workspace/translation_koenja/data/train/155.산업정보 연계 주요국 특허 영-한 데이터/эЪМьВм ы│╕ы╢А ы░П ъ▓╜ьШБ ь╗иьДдэМЕ ьДЬы╣ДьКдьЧЕ.json


эЪМьВм ы│╕ы╢А ы░П ъ▓╜ьШБ ь╗иьДдэМЕ ьДЬы╣ДьКдьЧЕ.json: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 64826.96it/s]

총 2160298 문장


### 4.2 en-ko validation data

In [94]:
file_list = sorted(glob(os.path.join(data_path, 'validation', '155.산업정보 연계 주요국 특허 영-한 데이터/*')))
file_list, len(file_list)

(['/data/heerak/workspace/translation_koenja/data/validation/155.산업정보 연계 주요국 특허 영-한 데이터/1ь░и ъ╕ИьЖНьаЬэТИ ы░П ъ╕ИьЖНъ┤Сым╝ ыПДыздьЧЕ.json',
  '/data/heerak/workspace/translation_koenja/data/validation/155.산업정보 연계 주요국 특허 영-한 데이터/ъ│Дый┤эЩЬьД▒ьаЬ ьаЬьб░ьЧЕ.json',
  '/data/heerak/workspace/translation_koenja/data/validation/155.산업정보 연계 주요국 특허 영-한 데이터/ъ│аым┤ ьЭШыеШ ы░П ъ╕░эГА ьЬДьГЭьЪй ы╣Дъ▓╜эЩФ ъ│аым┤ьаЬэТИ ьаЬьб░ьЧЕ.json',
  '/data/heerak/workspace/translation_koenja/data/validation/155.산업정보 연계 주요국 특허 영-한 데이터/ъ│аым┤ эГАьЭ┤ьЦ┤ ы░П эКЬы╕М ьГЭьВ░ьЧЕ.json',
  '/data/heerak/workspace/translation_koenja/data/validation/155.산업정보 연계 주요국 특허 영-한 데이터/ъ│аым┤ эМиэВ╣ыеШ ьаЬьб░ьЧЕ.json',
  '/data/heerak/workspace/translation_koenja/data/validation/155.산업정보 연계 주요국 특허 영-한 데이터/ъ│аым┤, эЩФэХЩьДмьЬа ы░П эФМыЭ╝ьКдэЛ▒ ьД▒эШХъ╕░ ьаЬьб░ьЧЕ.json',
  '/data/heerak/workspace/translation_koenja/data/validation/155.산업정보 연계 주요국 특허 영-한 데이터/ъ│бым╝ ъ░Аъ│╡эТИ ьаЬьб░ьЧЕ.json',
  '/data/heerak/workspace/translation_koenja/d

In [95]:
for fname in file_list:
    print(fname)
    source = 'en'
    target = 'ko'
    source_target = f'{source}_{target}'
    mode = 'valid'
    with open(fname, 'r') as json_file:
        json_data = json.load(json_file)
        
    if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')):
        with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 'r') as f:
            cnt = max(len(f.readlines()) - 1, 0)
    else:
        cnt = 0
    
    with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 
              'a' if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')) else 'w') as f:
        if cnt == 0:
            f.write(f'{source}\t{target}')
        for data in tqdm(json_data['labeled_data'], desc=f"{fname.split('/')[-1]}"):
            for key in ['invention_title', 'astrt_cont', 'claim']:
                en = data[f'{key}_eng'].replace('>', '').replace('\n', '').strip()
                ko = data[f'{key}_kor'].replace('>', '').replace('\n', '').strip()
                f.write(f'\n{en}\t{ko}')
                cnt += 1
    
    print(f'총 {cnt} 문장')

/data/heerak/workspace/translation_koenja/data/validation/155.산업정보 연계 주요국 특허 영-한 데이터/1ь░и ъ╕ИьЖНьаЬэТИ ы░П ъ╕ИьЖНъ┤Сым╝ ыПДыздьЧЕ.json


1ь░и ъ╕ИьЖНьаЬэТИ ы░П ъ╕ИьЖНъ┤Сым╝ ыПДыздьЧЕ.json: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [00:00<00:00, 29194.69it/s]


총 150047 문장
/data/heerak/workspace/translation_koenja/data/validation/155.산업정보 연계 주요국 특허 영-한 데이터/ъ│Дый┤эЩЬьД▒ьаЬ ьаЬьб░ьЧЕ.json


ъ│Дый┤эЩЬьД▒ьаЬ ьаЬьб░ьЧЕ.json: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 78/78 [00:00<00:00, 70235.23it/s]


총 150281 문장
/data/heerak/workspace/translation_koenja/data/validation/155.산업정보 연계 주요국 특허 영-한 데이터/ъ│аым┤ ьЭШыеШ ы░П ъ╕░эГА ьЬДьГЭьЪй ы╣Дъ▓╜эЩФ ъ│аым┤ьаЬэТИ ьаЬьб░ьЧЕ.json


...

총 269987 문장
/data/heerak/workspace/translation_koenja/data/validation/155.산업정보 연계 주요국 특허 영-한 데이터/эЩШъ▓╜ьДды╣Д ъ▒┤ьДдьЧЕ.json


эЩШъ▓╜ьДды╣Д ъ▒┤ьДдьЧЕ.json: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 17/17 [00:00<00:00, 40767.96it/s]


총 270038 문장
/data/heerak/workspace/translation_koenja/data/validation/155.산업정보 연계 주요국 특허 영-한 데이터/эЪМьВм ы│╕ы╢А ы░П ъ▓╜ьШБ ь╗иьДдэМЕ ьДЬы╣ДьКдьЧЕ.json


эЪМьВм ы│╕ы╢А ы░П ъ▓╜ьШБ ь╗иьДдэМЕ ьДЬы╣ДьКдьЧЕ.json: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:00<00:00, 21509.25it/s]

총 270044 문장


## 5. 156.전문분야 영-한, 중-한 번역 말뭉치(식품)

### 5.1 en-ko train data

In [96]:
file_list = sorted(glob(os.path.join(data_path, 'train', '156.전문분야 영-한, 중-한 번역 말뭉치(식품)/*')))
file_list, len(file_list)

(['/data/heerak/workspace/translation_koenja/data/train/156.전문분야 영-한, 중-한 번역 말뭉치(식품)/2-5_전문분야_식품_영한_train_set_1200000.json'],
 1)

In [97]:
for fname in file_list:
    print(fname)
    source = 'en'
    target = 'ko'
    source_target = f'{source}_{target}'
    mode = 'train'
    with open(fname, 'r') as json_file:
        json_data = json.load(json_file)
        
    if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')):
        with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 'r') as f:
            cnt = max(len(f.readlines()) - 1, 0)
    else:
        cnt = 0
    
    with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 
              'a' if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')) else 'w') as f:
        if cnt == 0:
            f.write(f'{source}\t{target}')
        for data in tqdm(json_data['data'], desc=f"{fname.split('/')[-1]}"):
            en = data['en'].replace('>', '').replace('\n', '').strip()
            ko = data['ko'].replace('>', '').replace('\n', '').strip()
            f.write(f'\n{en}\t{ko}')
            cnt += 1
    
    print(f'총 {cnt} 문장')

/data/heerak/workspace/translation_koenja/data/train/156.전문분야 영-한, 중-한 번역 말뭉치(식품)/2-5_전문분야_식품_영한_train_set_1200000.json


2-5_전문분야_식품_영한_train_set_1200000.json: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1200000/1200000 [00:02<00:00, 589177.77it/s]

총 3360298 문장


### 5.2 en-ko validation data

In [98]:
file_list = sorted(glob(os.path.join(data_path, 'validation', '156.전문분야 영-한, 중-한 번역 말뭉치(식품)/*')))
file_list, len(file_list)

(['/data/heerak/workspace/translation_koenja/data/validation/156.전문분야 영-한, 중-한 번역 말뭉치(식품)/2-5_전문분야_식품_영한_valid_set_150000.json'],
 1)

In [99]:
for fname in file_list:
    print(fname)
    source = 'en'
    target = 'ko'
    source_target = f'{source}_{target}'
    mode = 'valid'
    with open(fname, 'r') as json_file:
        json_data = json.load(json_file)
        
    if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')):
        with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 'r') as f:
            cnt = max(len(f.readlines()) - 1, 0)
    else:
        cnt = 0
    
    with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 
              'a' if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')) else 'w') as f:
        if cnt == 0:
            f.write(f'{source}\t{target}')
        for data in tqdm(json_data['data'], desc=f"{fname.split('/')[-1]}"):
            en = data['en'].replace('>', '').replace('\n', '').strip()
            ko = data['ko'].replace('>', '').replace('\n', '').strip()
            f.write(f'\n{en}\t{ko}')
            cnt += 1
    
    print(f'총 {cnt} 문장')

/data/heerak/workspace/translation_koenja/data/validation/156.전문분야 영-한, 중-한 번역 말뭉치(식품)/2-5_전문분야_식품_영한_valid_set_150000.json


2-5_전문분야_식품_영한_valid_set_150000.json: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 150000/150000 [00:00<00:00, 582564.34it/s]

총 420044 문장


## 6. 157.방송 콘텐츠 한-중, 한-일 번역 병렬 말뭉치 데이터

### 6.1 jp-ko train data

In [100]:
file_list = sorted(glob(os.path.join(data_path, 'train', '157.방송 콘텐츠 한-중, 한-일 번역 병렬 말뭉치 데이터/*일한*')))
file_list, len(file_list)

(['/data/heerak/workspace/translation_koenja/data/train/157.방송 콘텐츠 한-중, 한-일 번역 병렬 말뭉치 데이터/2-5_방송콘텐츠_일한_train_set_360000.json'],
 1)

In [101]:
for fname in file_list:
    print(fname)
    source = 'jp'
    target = 'ko'
    source_target = f'{source}_{target}'
    mode = 'train'
    with open(fname, 'r') as json_file:
        json_data = json.load(json_file)
        
    if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')):
        with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 'r') as f:
            cnt = max(len(f.readlines()) - 1, 0)
    else:
        cnt = 0
    
    with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 
              'a' if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')) else 'w') as f:
        if cnt == 0:
            f.write(f'{source}\t{target}')
        for data in tqdm(json_data['data'], desc=f"{fname.split('/')[-1]}"):
            jp = data['jp'].replace('>', '').replace('\n', '').strip()
            ko = data['ko'].replace('>', '').replace('\n', '').strip()
            f.write(f'\n{jp}\t{ko}')
            cnt += 1
    
    print(f'총 {cnt} 문장')

/data/heerak/workspace/translation_koenja/data/train/157.방송 콘텐츠 한-중, 한-일 번역 병렬 말뭉치 데이터/2-5_방송콘텐츠_일한_train_set_360000.json


2-5_방송콘텐츠_일한_train_set_360000.json: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 360000/360000 [00:00<00:00, 808244.50it/s]

총 960000 문장


### 6.2 jp-ko validation data

In [102]:
file_list = sorted(glob(os.path.join(data_path, 'validation', '157.방송 콘텐츠 한-중, 한-일 번역 병렬 말뭉치 데이터/*일한*')))
file_list, len(file_list)

(['/data/heerak/workspace/translation_koenja/data/validation/157.방송 콘텐츠 한-중, 한-일 번역 병렬 말뭉치 데이터/2-5_방송콘텐츠_일한_valid_set_45000.json'],
 1)

In [103]:
for fname in file_list:
    print(fname)
    source = 'jp'
    target = 'ko'
    source_target = f'{source}_{target}'
    mode = 'valid'
    with open(fname, 'r') as json_file:
        json_data = json.load(json_file)
        
    if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')):
        with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 'r') as f:
            cnt = max(len(f.readlines()) - 1, 0)
    else:
        cnt = 0
    
    with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 
              'a' if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')) else 'w') as f:
        if cnt == 0:
            f.write(f'{source}\t{target}')
        for data in tqdm(json_data['data'], desc=f"{fname.split('/')[-1]}"):
            jp = data['jp'].replace('>', '').replace('\n', '').strip()
            ko = data['ko'].replace('>', '').replace('\n', '').strip()
            f.write(f'\n{jp}\t{ko}')
            cnt += 1
    
    print(f'총 {cnt} 문장')

/data/heerak/workspace/translation_koenja/data/validation/157.방송 콘텐츠 한-중, 한-일 번역 병렬 말뭉치 데이터/2-5_방송콘텐츠_일한_valid_set_45000.json


2-5_방송콘텐츠_일한_valid_set_45000.json: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 45000/45000 [00:00<00:00, 792930.73it/s]

총 120000 문장


### 6.3 ko-jp train data

In [104]:
file_list = sorted(glob(os.path.join(data_path, 'train', '157.방송 콘텐츠 한-중, 한-일 번역 병렬 말뭉치 데이터/*한일*')))
file_list, len(file_list)

(['/data/heerak/workspace/translation_koenja/data/train/157.방송 콘텐츠 한-중, 한-일 번역 병렬 말뭉치 데이터/2-5_방송콘텐츠_한일_train_set_600000.json'],
 1)

In [105]:
for fname in file_list:
    print(fname)
    source = 'ko'
    target = 'jp'
    source_target = f'{source}_{target}'
    mode = 'train'
    with open(fname, 'r') as json_file:
        json_data = json.load(json_file)
        
    if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')):
        with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 'r') as f:
            cnt = max(len(f.readlines()) - 1, 0)
    else:
        cnt = 0
    
    with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 
              'a' if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')) else 'w') as f:
        if cnt == 0:
            f.write(f'{source}\t{target}')
        for data in tqdm(json_data['data'], desc=f"{fname.split('/')[-1]}"):
            jp = data['jp'].replace('>', '').replace('\n', '').strip()
            ko = data['ko'].replace('>', '').replace('\n', '').strip()
            f.write(f'\n{ko}\t{jp}')
            cnt += 1
    
    print(f'총 {cnt} 문장')

/data/heerak/workspace/translation_koenja/data/train/157.방송 콘텐츠 한-중, 한-일 번역 병렬 말뭉치 데이터/2-5_방송콘텐츠_한일_train_set_600000.json


2-5_방송콘텐츠_한일_train_set_600000.json: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 600000/600000 [00:00<00:00, 933889.68it/s]

총 1200000 문장


### 6.2 jp-ko validation data

In [106]:
file_list = sorted(glob(os.path.join(data_path, 'validation', '157.방송 콘텐츠 한-중, 한-일 번역 병렬 말뭉치 데이터/*한일*')))
file_list, len(file_list)

(['/data/heerak/workspace/translation_koenja/data/validation/157.방송 콘텐츠 한-중, 한-일 번역 병렬 말뭉치 데이터/2-5_방송콘텐츠_한일_valid_set_75000.json'],
 1)

In [107]:
for fname in file_list:
    print(fname)
    source = 'ko'
    target = 'jp'
    source_target = f'{source}_{target}'
    mode = 'valid'
    with open(fname, 'r') as json_file:
        json_data = json.load(json_file)
        
    if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')):
        with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 'r') as f:
            cnt = max(len(f.readlines()) - 1, 0)
    else:
        cnt = 0
    
    with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 
              'a' if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')) else 'w') as f:
        if cnt == 0:
            f.write(f'{source}\t{target}')
        for data in tqdm(json_data['data'], desc=f"{fname.split('/')[-1]}"):
            jp = data['jp'].replace('>', '').replace('\n', '').strip()
            ko = data['ko'].replace('>', '').replace('\n', '').strip()
            f.write(f'\n{ko}\t{jp}')
            cnt += 1
    
    print(f'총 {cnt} 문장')

/data/heerak/workspace/translation_koenja/data/validation/157.방송 콘텐츠 한-중, 한-일 번역 병렬 말뭉치 데이터/2-5_방송콘텐츠_한일_valid_set_75000.json


2-5_방송콘텐츠_한일_valid_set_75000.json: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 75000/75000 [00:00<00:00, 894796.31it/s]

총 150000 문장


## 7. 전문분야 한영 말뭉치

### 7.1 ko-en train data

In [108]:
file_list = sorted(glob(os.path.join(data_path, 'train', '전문분야 한영 말뭉치/ko2en*')))
file_list, len(file_list)

(['/data/heerak/workspace/translation_koenja/data/train/전문분야 한영 말뭉치/ko2en_edu_notice_1_training.csv',
  '/data/heerak/workspace/translation_koenja/data/train/전문분야 한영 말뭉치/ko2en_finance_1_training.csv',
  '/data/heerak/workspace/translation_koenja/data/train/전문분야 한영 말뭉치/ko2en_folk_and_food_1_training.csv',
  '/data/heerak/workspace/translation_koenja/data/train/전문분야 한영 말뭉치/ko2en_it_and_tech_1_training.csv',
  '/data/heerak/workspace/translation_koenja/data/train/전문분야 한영 말뭉치/ko2en_law_1_training.csv',
  '/data/heerak/workspace/translation_koenja/data/train/전문분야 한영 말뭉치/ko2en_medical_1_training.csv',
  '/data/heerak/workspace/translation_koenja/data/train/전문분야 한영 말뭉치/ko2en_sports_1_training.csv',
  '/data/heerak/workspace/translation_koenja/data/train/전문분야 한영 말뭉치/ko2en_travel_1_training.csv'],
 8)

In [109]:
for fname in file_list:
    print(fname)
    source = 'ko'
    target = 'en'
    source_target = f'{source}_{target}'
    mode = 'train'
    with open(fname, 'r') as csv_file:
        csv_data = csv.reader(csv_file)
        
        if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')):
            with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 'r') as f:
                cnt = max(len(f.readlines()) - 1, 0)
        else:
            cnt = 0

        with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 
                  'a' if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')) else 'w') as f:
            if cnt == 0:
                f.write(f'{source}\t{target}')
            for i, data in tqdm(enumerate(csv_data), desc=f"{fname.split('/')[-1]}"):
                if i == 0:
                    continue
                ko = data[2].replace('>', '').replace('\n', '').strip()
                en = data[3].replace('>', '').replace('\n', '').strip()
                f.write(f'\n{ko}\t{en}')
                cnt += 1

        print(f'총 {cnt} 문장')

/data/heerak/workspace/translation_koenja/data/train/전문분야 한영 말뭉치/ko2en_edu_notice_1_training.csv


ko2en_edu_notice_1_training.csv: 80001it [00:00, 214121.36it/s]


총 2480144 문장
/data/heerak/workspace/translation_koenja/data/train/전문분야 한영 말뭉치/ko2en_finance_1_training.csv


ko2en_finance_1_training.csv: 160001it [00:00, 195132.38it/s]


총 2640144 문장
/data/heerak/workspace/translation_koenja/data/train/전문분야 한영 말뭉치/ko2en_folk_and_food_1_training.csv


...

총 3280144 문장
/data/heerak/workspace/translation_koenja/data/train/전문분야 한영 말뭉치/ko2en_sports_1_training.csv


ko2en_sports_1_training.csv: 160001it [00:00, 214259.07it/s]


총 3440144 문장
/data/heerak/workspace/translation_koenja/data/train/전문분야 한영 말뭉치/ko2en_travel_1_training.csv


ko2en_travel_1_training.csv: 160001it [00:00, 225773.22it/s]

총 3600144 문장


### 7.2 ko-en validation data

In [110]:
file_list = sorted(glob(os.path.join(data_path, 'validation', '전문분야 한영 말뭉치/ko2en*')))
file_list, len(file_list)

(['/data/heerak/workspace/translation_koenja/data/validation/전문분야 한영 말뭉치/ko2en_edu_notice_2_validation.csv',
  '/data/heerak/workspace/translation_koenja/data/validation/전문분야 한영 말뭉치/ko2en_finance_2_validation.csv',
  '/data/heerak/workspace/translation_koenja/data/validation/전문분야 한영 말뭉치/ko2en_folk_and_food_2_validation.csv',
  '/data/heerak/workspace/translation_koenja/data/validation/전문분야 한영 말뭉치/ko2en_it_and_tech_2_validation.csv',
  '/data/heerak/workspace/translation_koenja/data/validation/전문분야 한영 말뭉치/ko2en_law_2_validation.csv',
  '/data/heerak/workspace/translation_koenja/data/validation/전문분야 한영 말뭉치/ko2en_medical_2_validation.csv',
  '/data/heerak/workspace/translation_koenja/data/validation/전문분야 한영 말뭉치/ko2en_sports_2_validation.csv',
  '/data/heerak/workspace/translation_koenja/data/validation/전문분야 한영 말뭉치/ko2en_travel_2_validation.csv'],
 8)

In [111]:
for fname in file_list:
    print(fname)
    source = 'ko'
    target = 'en'
    source_target = f'{source}_{target}'
    mode = 'valid'
    with open(fname, 'r') as csv_file:
        csv_data = csv.reader(csv_file)
        
        if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')):
            with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 'r') as f:
                cnt = max(len(f.readlines()) - 1, 0)
        else:
            cnt = 0

        with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 
                  'a' if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')) else 'w') as f:
            if cnt == 0:
                f.write(f'{source}\t{target}')
            for i, data in tqdm(enumerate(csv_data), desc=f"{fname.split('/')[-1]}"):
                if i == 0:
                    continue
                ko = data[2].replace('>', '').replace('\n', '').strip()
                en = data[3].replace('>', '').replace('\n', '').strip()
                f.write(f'\n{ko}\t{en}')
                cnt += 1

        print(f'총 {cnt} 문장')

/data/heerak/workspace/translation_koenja/data/validation/전문분야 한영 말뭉치/ko2en_edu_notice_2_validation.csv


ko2en_edu_notice_2_validation.csv: 10001it [00:00, 168341.10it/s]


총 310018 문장
/data/heerak/workspace/translation_koenja/data/validation/전문분야 한영 말뭉치/ko2en_finance_2_validation.csv


ko2en_finance_2_validation.csv: 20001it [00:00, 195472.33it/s]


총 330018 문장
/data/heerak/workspace/translation_koenja/data/validation/전문분야 한영 말뭉치/ko2en_folk_and_food_2_validation.csv


...

총 410018 문장
/data/heerak/workspace/translation_koenja/data/validation/전문분야 한영 말뭉치/ko2en_sports_2_validation.csv


ko2en_sports_2_validation.csv: 20001it [00:00, 44785.69it/s]


총 430018 문장
/data/heerak/workspace/translation_koenja/data/validation/전문분야 한영 말뭉치/ko2en_travel_2_validation.csv


ko2en_travel_2_validation.csv: 20001it [00:00, 214063.14it/s]

총 450018 문장


## 8. 한국어-영어 번역 말뭉치(기술과학)

### 8.1 ko-en train data

In [112]:
file_list = sorted(glob(os.path.join(data_path, 'train', '한국어-영어 번역 말뭉치(기술과학)/*')))
file_list, len(file_list)

(['/data/heerak/workspace/translation_koenja/data/train/한국어-영어 번역 말뭉치(기술과학)/1113_tech_train_set_1195228.json'],
 1)

In [113]:
for fname in file_list:
    print(fname)
    source = 'ko'
    target = 'en'
    source_target = f'{source}_{target}'
    mode = 'train'
    with open(fname, 'r') as json_file:
        json_data = json.load(json_file)
        
    if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')):
        with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 'r') as f:
            cnt = max(len(f.readlines()) - 1, 0)
    else:
        cnt = 0
    
    with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 
              'a' if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')) else 'w') as f:
        if cnt == 0:
            f.write(f'{source}\t{target}')
        for data in tqdm(json_data['data'], desc=f"{fname.split('/')[-1]}"):
            en = data['en'].replace('>', '').replace('\n', '').strip()
            ko = data['ko'].replace('>', '').replace('\n', '').strip()
            f.write(f'\n{ko}\t{en}')
            cnt += 1
    
    print(f'총 {cnt} 문장')

/data/heerak/workspace/translation_koenja/data/train/한국어-영어 번역 말뭉치(기술과학)/1113_tech_train_set_1195228.json


1113_tech_train_set_1195228.json: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1195228/1195228 [00:01<00:00, 651147.26it/s]

총 4795372 문장


### 8.2 ko-en validation data

In [114]:
file_list = sorted(glob(os.path.join(data_path, 'validation', '한국어-영어 번역 말뭉치(기술과학)/*')))
file_list

['/data/heerak/workspace/translation_koenja/data/validation/한국어-영어 번역 말뭉치(기술과학)/1113_tech_valid_set_149403.json']

In [115]:
for fname in file_list:
    print(fname)
    source = 'ko'
    target = 'en'
    source_target = f'{source}_{target}'
    mode = 'valid'
    with open(fname, 'r') as json_file:
        json_data = json.load(json_file)
        
    if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')):
        with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 'r') as f:
            cnt = max(len(f.readlines()) - 1, 0)
    else:
        cnt = 0
    
    with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 
              'a' if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')) else 'w') as f:
        if cnt == 0:
            f.write(f'{source}\t{target}')
        for data in tqdm(json_data['data'], desc=f"{fname.split('/')[-1]}"):
            en = data['en'].replace('>', '').replace('\n', '').strip()
            ko = data['ko'].replace('>', '').replace('\n', '').strip()
            f.write(f'\n{ko}\t{en}')
            cnt += 1
    
    print(f'총 {cnt} 문장')

/data/heerak/workspace/translation_koenja/data/validation/한국어-영어 번역 말뭉치(기술과학)/1113_tech_valid_set_149403.json


1113_tech_valid_set_149403.json: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 149403/149403 [00:00<00:00, 652443.80it/s]

총 599421 문장


## 9. 한국어-영어 번역 말뭉치(사회과학)

### 9.1 ko-en train data

In [116]:
file_list = sorted(glob(os.path.join(data_path, 'train', '한국어-영어 번역 말뭉치(사회과학)/*')))
file_list, len(file_list)

(['/data/heerak/workspace/translation_koenja/data/train/한국어-영어 번역 말뭉치(사회과학)/1113_social_train_set_1210529.json'],
 1)

In [117]:
for fname in file_list:
    print(fname)
    source = 'ko'
    target = 'en'
    source_target = f'{source}_{target}'
    mode = 'train'
    with open(fname, 'r') as json_file:
        json_data = json.load(json_file)
        
    if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')):
        with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 'r') as f:
            cnt = max(len(f.readlines()) - 1, 0)
    else:
        cnt = 0
    
    with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 
              'a' if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')) else 'w') as f:
        if cnt == 0:
            f.write(f'{source}\t{target}')
        for data in tqdm(json_data['data'], desc=f"{fname.split('/')[-1]}"):
            en = data['en'].replace('>', '').replace('\n', '').strip()
            ko = data['ko'].replace('>', '').replace('\n', '').strip()
            f.write(f'\n{ko}\t{en}')
            cnt += 1
    
    print(f'총 {cnt} 문장')

/data/heerak/workspace/translation_koenja/data/train/한국어-영어 번역 말뭉치(사회과학)/1113_social_train_set_1210529.json


1113_social_train_set_1210529.json: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1210529/1210529 [00:01<00:00, 619683.13it/s]

총 6005901 문장


### 9.2 ko-en validation data

In [118]:
file_list = sorted(glob(os.path.join(data_path, 'validation', '한국어-영어 번역 말뭉치(사회과학)/*')))
file_list

['/data/heerak/workspace/translation_koenja/data/validation/한국어-영어 번역 말뭉치(사회과학)/1113_social_valid_set_151316.json']

In [119]:
for fname in file_list:
    print(fname)
    source = 'ko'
    target = 'en'
    source_target = f'{source}_{target}'
    mode = 'valid'
    with open(fname, 'r') as json_file:
        json_data = json.load(json_file)
        
    if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')):
        with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 'r') as f:
            cnt = max(len(f.readlines()) - 1, 0)
    else:
        cnt = 0
    
    with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 
              'a' if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')) else 'w') as f:
        if cnt == 0:
            f.write(f'{source}\t{target}')
        for data in tqdm(json_data['data'], desc=f"{fname.split('/')[-1]}"):
            en = data['en'].replace('>', '').replace('\n', '').strip()
            ko = data['ko'].replace('>', '').replace('\n', '').strip()
            f.write(f'\n{ko}\t{en}')
            cnt += 1
    
    print(f'총 {cnt} 문장')

/data/heerak/workspace/translation_koenja/data/validation/한국어-영어 번역 말뭉치(사회과학)/1113_social_valid_set_151316.json


1113_social_valid_set_151316.json: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 151316/151316 [00:00<00:00, 614594.49it/s]

총 750737 문장


## 10. 한국어-영어 번역(병렬) 말뭉치

### 10.1 ko-en train data

In [120]:
file_list = sorted(glob(os.path.join(data_path, 'train', '한국어-영어 번역(병렬) 말뭉치/*')))
file_list

['/data/heerak/workspace/translation_koenja/data/train/한국어-영어 번역(병렬) 말뭉치/1_구어체(1).xlsx',
 '/data/heerak/workspace/translation_koenja/data/train/한국어-영어 번역(병렬) 말뭉치/1_구어체(2).xlsx',
 '/data/heerak/workspace/translation_koenja/data/train/한국어-영어 번역(병렬) 말뭉치/2_대화체.xlsx',
 '/data/heerak/workspace/translation_koenja/data/train/한국어-영어 번역(병렬) 말뭉치/3_문어체_뉴스(1).xlsx',
 '/data/heerak/workspace/translation_koenja/data/train/한국어-영어 번역(병렬) 말뭉치/3_문어체_뉴스(2).xlsx',
 '/data/heerak/workspace/translation_koenja/data/train/한국어-영어 번역(병렬) 말뭉치/3_문어체_뉴스(3).xlsx',
 '/data/heerak/workspace/translation_koenja/data/train/한국어-영어 번역(병렬) 말뭉치/3_문어체_뉴스(4).xlsx',
 '/data/heerak/workspace/translation_koenja/data/train/한국어-영어 번역(병렬) 말뭉치/4_문어체_한국문화.xlsx',
 '/data/heerak/workspace/translation_koenja/data/train/한국어-영어 번역(병렬) 말뭉치/5_문어체_조례.xlsx',
 '/data/heerak/workspace/translation_koenja/data/train/한국어-영어 번역(병렬) 말뭉치/6_문어체_지자체웹사이트.xlsx']

In [121]:
for fname in file_list:
    print(fname)
    source = 'ko'
    target = 'en'
    source_target = f'{source}_{target}'
    mode = 'train'
    df = pd.read_excel(fname)
        
    if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')):
        with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 'r') as f:
            cnt = max(len(f.readlines()) - 1, 0)
    else:
        cnt = 0
    
    with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 
              'a' if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')) else 'w') as f:
        if cnt == 0:
            f.write(f'{source}\t{target}')
        for idx, data in tqdm(df.iterrows(), total=len(df), desc=f"{fname.split('/')[-1]}"):
            ko = data['원문'].replace('>', '').replace('\n', '').strip()
            en = data['번역문'].replace('>', '').replace('\n', '').strip()
            f.write(f'\n{ko}\t{en}')
            cnt += 1
    
    print(f'총 {cnt} 문장')

/data/heerak/workspace/translation_koenja/data/train/한국어-영어 번역(병렬) 말뭉치/1_구어체(1).xlsx


1_구어체(1).xlsx: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200000/200000 [00:07<00:00, 27534.72it/s]


총 6205901 문장
/data/heerak/workspace/translation_koenja/data/train/한국어-영어 번역(병렬) 말뭉치/1_구어체(2).xlsx


/home/heerak/anaconda3/envs/heerak/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
1_구어체(2).xlsx: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 200000/200000 [00:07<00:00, 27251.57it/s]


총 6405901 문장
/data/heerak/workspace/translation_koenja/data/train/한국어-영어 번역(병렬) 말뭉치/2_대화체.xlsx


...

총 7407934 문장
/data/heerak/workspace/translation_koenja/data/train/한국어-영어 번역(병렬) 말뭉치/5_문어체_조례.xlsx


/home/heerak/anaconda3/envs/heerak/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
5_문어체_조례.xlsx: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100298/100298 [00:03<00:00, 26411.78it/s]


총 7508232 문장
/data/heerak/workspace/translation_koenja/data/train/한국어-영어 번역(병렬) 말뭉치/6_문어체_지자체웹사이트.xlsx


/home/heerak/anaconda3/envs/heerak/lib/python3.8/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
6_문어체_지자체웹사이트.xlsx: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100087/100087 [00:03<00:00, 26571.05it/s]

총 7608319 문장


## 11. 한국어-일본어 번역 말뭉치

### 11.1 ko-jp train data

In [122]:
file_list = sorted(glob(os.path.join(data_path, 'train', '한국어-일본어 번역 말뭉치/ko2ja*')))
file_list, len(file_list)

(['/data/heerak/workspace/translation_koenja/data/train/한국어-일본어 번역 말뭉치/ko2ja_auto_1_training.json',
  '/data/heerak/workspace/translation_koenja/data/train/한국어-일본어 번역 말뭉치/ko2ja_edu_1_training.json',
  '/data/heerak/workspace/translation_koenja/data/train/한국어-일본어 번역 말뭉치/ko2ja_finance_1_training.json',
  '/data/heerak/workspace/translation_koenja/data/train/한국어-일본어 번역 말뭉치/ko2ja_folk_1_training.json',
  '/data/heerak/workspace/translation_koenja/data/train/한국어-일본어 번역 말뭉치/ko2ja_it_tech_1_training.json',
  '/data/heerak/workspace/translation_koenja/data/train/한국어-일본어 번역 말뭉치/ko2ja_kpop_1_training.json',
  '/data/heerak/workspace/translation_koenja/data/train/한국어-일본어 번역 말뭉치/ko2ja_medical_1_training.json',
  '/data/heerak/workspace/translation_koenja/data/train/한국어-일본어 번역 말뭉치/ko2ja_patent_1_training.json',
  '/data/heerak/workspace/translation_koenja/data/train/한국어-일본어 번역 말뭉치/ko2ja_society_1_training.json'],
 9)

In [123]:
for fname in file_list:
    print(fname)
    source = 'ko'
    target = 'jp'
    source_target = f'{source}_{target}'
    mode = 'train'
    with open(fname, 'r') as json_file:
        json_data = json.load(json_file)
        
    if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')):
        with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 'r') as f:
            cnt = max(len(f.readlines()) - 1, 0)
    else:
        cnt = 0
    
    with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 
              'a' if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')) else 'w') as f:
        if cnt == 0:
            f.write(f'{source}\t{target}')
        for data in tqdm(json_data, desc=f"{fname.split('/')[-1]}"):
            ko = data['한국어'].replace('>', '').replace('\n', '').strip()
            jp = data['일본어'].replace('>', '').replace('\n', '').strip()
            f.write(f'\n{ko}\t{jp}')
            cnt += 1

        print(f'총 {cnt} 문장')

/data/heerak/workspace/translation_koenja/data/train/한국어-일본어 번역 말뭉치/ko2ja_auto_1_training.json


ko2ja_auto_1_training.json: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 160000/160000 [00:00<00:00, 696632.77it/s]


총 1360000 문장
/data/heerak/workspace/translation_koenja/data/train/한국어-일본어 번역 말뭉치/ko2ja_edu_1_training.json


ko2ja_edu_1_training.json: 100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 80000/80000 [00:00<00:00, 729912.27it/s]


총 1440000 문장
/data/heerak/workspace/translation_koenja/data/train/한국어-일본어 번역 말뭉치/ko2ja_finance_1_training.json


...

총 2120000 문장
/data/heerak/workspace/translation_koenja/data/train/한국어-일본어 번역 말뭉치/ko2ja_patent_1_training.json


ko2ja_patent_1_training.json: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 120000/120000 [00:00<00:00, 650077.28it/s]


총 2240000 문장
/data/heerak/workspace/translation_koenja/data/train/한국어-일본어 번역 말뭉치/ko2ja_society_1_training.json


ko2ja_society_1_training.json: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 160000/160000 [00:00<00:00, 682083.14it/s]

총 2400000 문장


### 11.2 ko-jp validation data

In [124]:
file_list = sorted(glob(os.path.join(data_path, 'validation', '한국어-일본어 번역 말뭉치/ko2ja*')))
file_list, len(file_list)

(['/data/heerak/workspace/translation_koenja/data/validation/한국어-일본어 번역 말뭉치/ko2ja_auto_2_validation.json',
  '/data/heerak/workspace/translation_koenja/data/validation/한국어-일본어 번역 말뭉치/ko2ja_edu_2_validation.json',
  '/data/heerak/workspace/translation_koenja/data/validation/한국어-일본어 번역 말뭉치/ko2ja_finance_2_validation.json',
  '/data/heerak/workspace/translation_koenja/data/validation/한국어-일본어 번역 말뭉치/ko2ja_folk_2_validation.json',
  '/data/heerak/workspace/translation_koenja/data/validation/한국어-일본어 번역 말뭉치/ko2ja_it_tech_2_validation.json',
  '/data/heerak/workspace/translation_koenja/data/validation/한국어-일본어 번역 말뭉치/ko2ja_kpop_2_validation.json',
  '/data/heerak/workspace/translation_koenja/data/validation/한국어-일본어 번역 말뭉치/ko2ja_medical_2_validation.json',
  '/data/heerak/workspace/translation_koenja/data/validation/한국어-일본어 번역 말뭉치/ko2ja_patent_2_validation.json',
  '/data/heerak/workspace/translation_koenja/data/validation/한국어-일본어 번역 말뭉치/ko2ja_society_2_validation.json'],
 9)

In [125]:
for fname in file_list:
    print(fname)
    source = 'ko'
    target = 'jp'
    source_target = f'{source}_{target}'
    mode = 'valid'
    with open(fname, 'r') as json_file:
        json_data = json.load(json_file)
        
    if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')):
        with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 'r') as f:
            cnt = max(len(f.readlines()) - 1, 0)
    else:
        cnt = 0
    
    with open(os.path.join(data_path, f'{source_target}/{mode}.tsv'), 
              'a' if os.path.isfile(os.path.join(data_path, f'{source_target}/{mode}.tsv')) else 'w') as f:
        if cnt == 0:
            f.write(f'{source}\t{target}')
        for data in tqdm(json_data, desc=f"{fname.split('/')[-1]}"):
            ko = data['한국어'].replace('>', '').replace('\n', '').strip()
            jp = data['일본어'].replace('>', '').replace('\n', '').strip()
            f.write(f'\n{ko}\t{jp}')
            cnt += 1

        print(f'총 {cnt} 문장')

/data/heerak/workspace/translation_koenja/data/validation/한국어-일본어 번역 말뭉치/ko2ja_auto_2_validation.json


ko2ja_auto_2_validation.json: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20000/20000 [00:00<00:00, 692964.13it/s]


총 170000 문장
/data/heerak/workspace/translation_koenja/data/validation/한국어-일본어 번역 말뭉치/ko2ja_edu_2_validation.json


ko2ja_edu_2_validation.json: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 618774.93it/s]


총 180000 문장
/data/heerak/workspace/translation_koenja/data/validation/한국어-일본어 번역 말뭉치/ko2ja_finance_2_validation.json


...

총 265000 문장
/data/heerak/workspace/translation_koenja/data/validation/한국어-일본어 번역 말뭉치/ko2ja_patent_2_validation.json


ko2ja_patent_2_validation.json: 100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 15000/15000 [00:00<00:00, 608263.90it/s]


총 280000 문장
/data/heerak/workspace/translation_koenja/data/validation/한국어-일본어 번역 말뭉치/ko2ja_society_2_validation.json


ko2ja_society_2_validation.json: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 20000/20000 [00:00<00:00, 635240.73it/s]

총 300000 문장


# Huggingface datasets Upload

## 1. Huggingface_hub Login

In [62]:
notebook_login()

## 2. datasets push_to_hub

In [11]:
for i, (source, target) in enumerate([['ko', 'en'], ['en', 'ko'], ['ko', 'jp'], ['jp', 'ko']]):
    datasets = DatasetDict()
    for mode in ['train', 'valid']:
        source_target = f'{source}_{target}'

        print(f'{mode} | {source_target} | 데이터')

        source_list = []
        target_list = []

        with open(os.path.join(data_path, source_target, f'{mode}.tsv'), 'r') as f:
            lines = f.readlines()
            for idx, line in tqdm(enumerate(lines), total=len(lines)):
                if idx == 0:
                    continue
                split_source_target = line.replace('\n', '').split('\t')
                if len(split_source_target) != 2:
                    continue
                source_list.append(split_source_target[0])
                target_list.append(split_source_target[1])

            print('전체 병렬 데이터 길이:', idx + 1)  # header 포함
            print('source_list 길이:', len(source_list))
            print('target_list 길이:', len(target_list))
            
        df = pd.DataFrame()
        df[source] = source_list
        df[target] = target_list
        df.drop_duplicates(inplace=True)
        
        datasets[mode] = Dataset.from_pandas(df)
    
    datasets.push_to_hub(f"Heerak/{source}_{target}_translation")
    create_tag(f"Heerak/{source}_{target}_translation", tag="1.0", repo_type="dataset")

train | en_ko | 데이터


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3360299/3360299 [00:03<00:00, 928393.70it/s]


전체 병렬 데이터 길이: 3360299
source_list 길이: 3360296
target_list 길이: 3360296


,en,ko
0,"I'm glad to hear that, and I hope you do consi...","그 말을 들으니 기쁘고, 저희와 거래하는 것을 고려해 주셨으면 합니다."
1,"I'm definitely thinking about it, but I have s...","확실히 생각하고 있습니다만, 몇 가지 여쭤보고 싶은 게 있어요."
2,"In today's world, one in every five families h...",오늘날 세계 5가구 중 1가구는 고양이나 개 또는 둘 다를 키우고 있습니다.
3,"When you tell them, we'll take care of their c...","그들에게 말하면, 저희가 그 아이들을 돌볼 것입니다."
4,"OK, how about for swimming?","좋아요, 수영은 어떤가요?"


valid | en_ko | 데이터


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 420045/420045 [00:00<00:00, 927961.64it/s]


전체 병렬 데이터 길이: 420045
source_list 길이: 420044
target_list 길이: 420044


,en,ko
0,"Features include heart rate monitoring, 14-day...","심박수 모니터링, 14일 배터리 수명, 음악 플레이 제어 디스플레이 수면 및 수영 ..."
1,The touch screen has 3rd generation tempered g...,터치스크린에는 3세대 강화유리와 지문 방지 코팅이 적용되었습니다.
2,"For product inquiries, please send us an email.",제품 문의는 이메일로 주시면 감사하겠습니다.
3,"Hello, BBB company AAA.","안녕하세요, BBB회사 AAA입니다."
4,I would like to introduce the Ultimate Radianc...,저는 얼티밋 래디언스 컬렉션 스킨케어 선물 세트 제품을 소개하려고 합니다.
